<a href="https://colab.research.google.com/github/kappandrew2/DataPreProcessing/blob/main/MarketResearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from sklearn.model_selection import train_test_split

In [15]:
#Create CSV from data export
#https://www.wsj.com/market-data/quotes/index/SPX/historical-prices

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SandPHistoricalPrices2013ToMid2021.csv')

print(dataset)

Mounted at /content/drive
          Date    Open      High      Low     Close     Volume
0     12/31/20  371.78  374.6600  371.232  373.8800   78520703
1     12/30/20  372.34  373.1000  371.570  371.9900   49455262
2     12/29/20  373.81  374.0000  370.830  371.4600   53680449
3     12/28/20  371.74  372.5900  371.070  372.1700   39000398
4     12/24/20  368.08  369.0300  367.450  369.0000   26457850
...        ...     ...       ...      ...       ...        ...
2010  01/08/13  145.71  145.9100  144.980  145.5496  121265102
2011  01/07/13  145.85  146.4442  145.430  145.9695  110002398
2012  01/04/13  145.97  146.6100  145.670  146.3700  116817695
2013  01/03/13  145.99  146.3700  145.340  145.7341  144761797
2014  01/02/13  145.11  146.1500  144.730  146.0600  192058891

[2015 rows x 6 columns]


In [16]:
dataset = dataset.drop([' Open', ' High', ' Low', ' Volume'], axis = 1)

dataset.rename({' Close': 'Close'}, axis=1, inplace = True)


In [17]:
dataset['Date'] = pd.to_datetime(dataset['Date'])

dataset.dtypes

Date     datetime64[ns]
Close           float64
dtype: object

In [18]:
dataset['DOW'] = dataset['Date'].dt.dayofweek
dataset['DOY'] = dataset['Date'].dt.dayofyear
dataset['Week'] = dataset['Date'].dt.week
dataset['Month'] = dataset['Date'].dt.month
dataset['Quarter'] = dataset['Date'].dt.quarter

dataset.dtypes

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Date       datetime64[ns]
Close             float64
DOW                 int64
DOY                 int64
Week                int64
Month               int64
Quarter             int64
dtype: object

In [31]:
dataset['gain_loss-0'] = dataset['Close'].diff(-1)
dataset['gain_loss-1'] = dataset['Close'].diff(-2)
dataset['gain_loss-1'] = dataset['gain_loss-1'].shift(periods=-1, fill_value=0)
dataset['gain_loss-2'] = dataset['Close'].diff(-3) 
dataset['gain_loss-2'] = dataset['gain_loss-2'].shift(periods=-1, fill_value=0)
dataset['gain_loss-3'] = dataset['Close'].diff(-4) 
dataset['gain_loss-3'] = dataset['gain_loss-3'].shift(periods=-1, fill_value=0)
dataset['gain_loss-4'] = dataset['Close'].diff(-5) 
dataset['gain_loss-4'] = dataset['gain_loss-4'].shift(periods=-1, fill_value=0)


dataset.dtypes

Date            datetime64[ns]
Close                  float64
DOW                      int64
DOY                      int64
Week                     int64
Month                    int64
Quarter                  int64
gain_loss-0            float64
gain_loss-1            float64
gain_loss-2            float64
gain_loss-3            float64
gain_loss-4            float64
gain_loss-0b             int64
gain_loss-1b             int64
gain_loss-2b             int64
gain_loss-3b             int64
gain_loss-4b             int64
prior_day-1            float64
prior_day-2            float64
dtype: object

In [32]:
dataset['gain_loss-0b'] = np.where(dataset['gain_loss-0'] > 0, 1, 0)
dataset['gain_loss-1b'] = np.where(dataset['gain_loss-1'] > 0, 1, 0)
dataset['gain_loss-2b'] = np.where(dataset['gain_loss-2'] > 0, 1, 0)
dataset['gain_loss-3b'] = np.where(dataset['gain_loss-3'] > 0, 1, 0)
dataset['gain_loss-4b'] = np.where(dataset['gain_loss-4'] > 0, 1, 0)

dataset.dtypes

Date            datetime64[ns]
Close                  float64
DOW                      int64
DOY                      int64
Week                     int64
Month                    int64
Quarter                  int64
gain_loss-0            float64
gain_loss-1            float64
gain_loss-2            float64
gain_loss-3            float64
gain_loss-4            float64
gain_loss-0b             int64
gain_loss-1b             int64
gain_loss-2b             int64
gain_loss-3b             int64
gain_loss-4b             int64
prior_day-1            float64
prior_day-2            float64
dtype: object

In [41]:
dataset['gain_loss-total'] = dataset['gain_loss-1b'] + dataset['gain_loss-2b'] + dataset['gain_loss-3b'] + dataset['gain_loss-4b']

dataset.head()

,Date,Close,DOW,DOY,Week,Month,Quarter,gain_loss-0,gain_loss-1,gain_loss-2,gain_loss-3,gain_loss-4,gain_loss-0b,gain_loss-1b,gain_loss-2b,gain_loss-3b,gain_loss-4b,prior_day-1,prior_day-2,prior_day-3,prior_day-4,prior_day-5,prior_day-1b,prior_day-2b,prior_day-3b,prior_day-4b,prior_day-5b,prior_day-total,gain_loss-total
0,2020-12-31,373.88,3,366,53,12,4,1.89,-0.18,2.99,4.42,4.75,1,0,1,1,1,0.53,-0.71,3.17,1.43,0.33,1,0,1,1,1,4,3
1,2020-12-30,371.99,2,365,53,12,4,0.53,2.46,3.89,4.22,3.60,1,1,1,1,1,-0.71,3.17,1.43,0.33,-0.62,0,1,1,1,0,3,4
2,2020-12-29,371.46,1,364,53,12,4,-0.71,4.60,4.93,4.31,2.99,0,1,1,1,1,3.17,1.43,0.33,-0.62,-1.32,1,1,1,0,0,3,4
3,2020-12-28,372.17,0,363,53,12,4,3.17,1.76,1.14,-0.18,-3.24,1,1,1,0,0,1.43,0.33,-0.62,-1.32,-3.06,1,1,0,0,0,2,2
4,2020-12-24,369.00,3,359,52,12,4,1.43,-0.29,-1.61,-4.67,-2.60,1,0,0,0,0,0.33,-0.62,-1.32,-3.06,2.07,1,0,0,0,1,2,0


In [36]:
dataset['prior_day-1'] = dataset['gain_loss-0']
dataset['prior_day-1'] = dataset['prior_day-1'].shift(periods=-1, fill_value=0)
dataset['prior_day-2'] = dataset['gain_loss-0']
dataset['prior_day-2'] = dataset['prior_day-2'].shift(periods=-2, fill_value=0)
dataset['prior_day-3'] = dataset['gain_loss-0']
dataset['prior_day-3'] = dataset['prior_day-3'].shift(periods=-3, fill_value=0)
dataset['prior_day-4'] = dataset['gain_loss-0']
dataset['prior_day-4'] = dataset['prior_day-4'].shift(periods=-4, fill_value=0)
dataset['prior_day-5'] = dataset['gain_loss-0']
dataset['prior_day-5'] = dataset['prior_day-5'].shift(periods=-5, fill_value=0)
dataset.head()

,Date,Close,DOW,DOY,Week,Month,Quarter,gain_loss-0,gain_loss-1,gain_loss-2,gain_loss-3,gain_loss-4,gain_loss-0b,gain_loss-1b,gain_loss-2b,gain_loss-3b,gain_loss-4b,prior_day-1,prior_day-2,prior_day-3,prior_day-4,prior_day-5
0,2020-12-31,373.88,3,366,53,12,4,1.89,-0.18,2.99,4.42,4.75,1,0,1,1,1,0.53,-0.71,3.17,1.43,0.33
1,2020-12-30,371.99,2,365,53,12,4,0.53,2.46,3.89,4.22,3.60,1,1,1,1,1,-0.71,3.17,1.43,0.33,-0.62
2,2020-12-29,371.46,1,364,53,12,4,-0.71,4.60,4.93,4.31,2.99,0,1,1,1,1,3.17,1.43,0.33,-0.62,-1.32
3,2020-12-28,372.17,0,363,53,12,4,3.17,1.76,1.14,-0.18,-3.24,1,1,1,0,0,1.43,0.33,-0.62,-1.32,-3.06
4,2020-12-24,369.00,3,359,52,12,4,1.43,-0.29,-1.61,-4.67,-2.60,1,0,0,0,0,0.33,-0.62,-1.32,-3.06,2.07


In [38]:
dataset['prior_day-1b'] = np.where(dataset['prior_day-1'] > 0, 1, 0)
dataset['prior_day-2b'] = np.where(dataset['prior_day-2'] > 0, 1, 0)
dataset['prior_day-3b'] = np.where(dataset['prior_day-3'] > 0, 1, 0)
dataset['prior_day-4b'] = np.where(dataset['prior_day-4'] > 0, 1, 0)
dataset['prior_day-5b'] = np.where(dataset['prior_day-5'] > 0, 1, 0)
dataset.head()

,Date,Close,DOW,DOY,Week,Month,Quarter,gain_loss-0,gain_loss-1,gain_loss-2,gain_loss-3,gain_loss-4,gain_loss-0b,gain_loss-1b,gain_loss-2b,gain_loss-3b,gain_loss-4b,prior_day-1,prior_day-2,prior_day-3,prior_day-4,prior_day-5,prior_day-1b,prior_day-2b,prior_day-3b,prior_day-4b,prior_day-5b
0,2020-12-31,373.88,3,366,53,12,4,1.89,-0.18,2.99,4.42,4.75,1,0,1,1,1,0.53,-0.71,3.17,1.43,0.33,1,0,1,1,1
1,2020-12-30,371.99,2,365,53,12,4,0.53,2.46,3.89,4.22,3.60,1,1,1,1,1,-0.71,3.17,1.43,0.33,-0.62,0,1,1,1,0
2,2020-12-29,371.46,1,364,53,12,4,-0.71,4.60,4.93,4.31,2.99,0,1,1,1,1,3.17,1.43,0.33,-0.62,-1.32,1,1,1,0,0
3,2020-12-28,372.17,0,363,53,12,4,3.17,1.76,1.14,-0.18,-3.24,1,1,1,0,0,1.43,0.33,-0.62,-1.32,-3.06,1,1,0,0,0
4,2020-12-24,369.00,3,359,52,12,4,1.43,-0.29,-1.61,-4.67,-2.60,1,0,0,0,0,0.33,-0.62,-1.32,-3.06,2.07,1,0,0,0,1


In [42]:
dataset['prior_day-total'] = dataset['prior_day-1b'] + dataset['prior_day-2b'] + dataset['prior_day-3b'] + dataset['prior_day-4b'] + dataset['prior_day-5b'] 
dataset.head()

,Date,Close,DOW,DOY,Week,Month,Quarter,gain_loss-0,gain_loss-1,gain_loss-2,gain_loss-3,gain_loss-4,gain_loss-0b,gain_loss-1b,gain_loss-2b,gain_loss-3b,gain_loss-4b,prior_day-1,prior_day-2,prior_day-3,prior_day-4,prior_day-5,prior_day-1b,prior_day-2b,prior_day-3b,prior_day-4b,prior_day-5b,prior_day-total,gain_loss-total
0,2020-12-31,373.88,3,366,53,12,4,1.89,-0.18,2.99,4.42,4.75,1,0,1,1,1,0.53,-0.71,3.17,1.43,0.33,1,0,1,1,1,4,3
1,2020-12-30,371.99,2,365,53,12,4,0.53,2.46,3.89,4.22,3.60,1,1,1,1,1,-0.71,3.17,1.43,0.33,-0.62,0,1,1,1,0,3,4
2,2020-12-29,371.46,1,364,53,12,4,-0.71,4.60,4.93,4.31,2.99,0,1,1,1,1,3.17,1.43,0.33,-0.62,-1.32,1,1,1,0,0,3,4
3,2020-12-28,372.17,0,363,53,12,4,3.17,1.76,1.14,-0.18,-3.24,1,1,1,0,0,1.43,0.33,-0.62,-1.32,-3.06,1,1,0,0,0,2,2
4,2020-12-24,369.00,3,359,52,12,4,1.43,-0.29,-1.61,-4.67,-2.60,1,0,0,0,0,0.33,-0.62,-1.32,-3.06,2.07,1,0,0,0,1,2,0


In [50]:
dataset['prior_day-1_roll_avg_30d'] = dataset['prior_day-1'].rolling(10).mean()

#create a new df that has a reverse index then concant it back to 

dataset.head()

,Date,Close,DOW,DOY,Week,Month,Quarter,gain_loss-0,gain_loss-1,gain_loss-2,gain_loss-3,gain_loss-4,gain_loss-0b,gain_loss-1b,gain_loss-2b,gain_loss-3b,gain_loss-4b,prior_day-1,prior_day-2,prior_day-3,prior_day-4,prior_day-5,prior_day-1b,prior_day-2b,prior_day-3b,prior_day-4b,prior_day-5b,prior_day-total,gain_loss-total,prior_day-1_roll_avg_30d
0,2020-12-31,373.88,3,366,53,12,4,1.89,-0.18,2.99,4.42,4.75,1,0,1,1,1,0.53,-0.71,3.17,1.43,0.33,1,0,1,1,1,4,3,NaN
1,2020-12-30,371.99,2,365,53,12,4,0.53,2.46,3.89,4.22,3.60,1,1,1,1,1,-0.71,3.17,1.43,0.33,-0.62,0,1,1,1,0,3,4,NaN
2,2020-12-29,371.46,1,364,53,12,4,-0.71,4.60,4.93,4.31,2.99,0,1,1,1,1,3.17,1.43,0.33,-0.62,-1.32,1,1,1,0,0,3,4,NaN
3,2020-12-28,372.17,0,363,53,12,4,3.17,1.76,1.14,-0.18,-3.24,1,1,1,0,0,1.43,0.33,-0.62,-1.32,-3.06,1,1,0,0,0,2,2,NaN
4,2020-12-24,369.00,3,359,52,12,4,1.43,-0.29,-1.61,-4.67,-2.60,1,0,0,0,0,0.33,-0.62,-1.32,-3.06,2.07,1,0,0,0,1,2,0,NaN
